# 들어가기에 앞서 
 - 이번 프로젝트때 [1]트위터 스크래핑을 통한 데이터만을 썻음

In [ ]:
!pip install facebook-sdk
!pip install selenium
!pip install bs4
!pip install lxml
!pip install twitterscraper
!pip install openpyxl
!pip install python-twitter

In [ ]:

import sys
from urllib.request import Request, urlopen
from datetime import *
import json
import requests as req

#페북,인스타,트위터
import facebook
import twitter
#셀레니움, 뷰티풀숲
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
#넘피,판다스,멧플로립
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##  1. 주식종목 불러오기
    - 주식종목에서 기업명만 추출 -> 크롤링 키워드로 사용

In [ ]:
#증권사 API에서 가져운 주식종목불러오기
train = pd.read_csv('./종목명v7.csv', encoding = 'euc-kr')

In [ ]:

#불러온 주식종목에서 기업명만을 추출해내기위한 작업
nameList = [ "1우B",
    "1우","2우B","2우","3우B","3우C","4우B","4우","우B","전환", ")","(",
    "(합성 H)","(합성)","선물","인버스","채권주식","국채","코스닥","나스닥",
    "200","300","100","대형고배당","TR","150","미국채","미국","가중",
    "TOP10","","Enhanced",
    "제11호스팩","제12호스팩","제13호스팩","제14호스팩","제15호스팩","제16호스팩",
    "제17호스팩","제18호스팩","제19호스팩",
    "제1호스팩","제1호스팩","제3호스팩","제4호스팩","제5호스팩","제6호스팩",
    "제7호스팩","제8호스팩","제9호스팩","제10호스팩","7호스팩", "스팩5호",
           "2호스팩","4호스팩","5호스팩","6호스팩","8호스팩","9호스팩","10호스팩",
"14호스팩","16호스팩","19호스팩","20호스팩"]
for i in range(len(nameList)):
    train['종목명'] = train['종목명'].str.replace(nameList[i], '')
train.iloc[7,2]

# 종목명뒤에 '우' 탈락을 위한 조건문
for j in range(train['종목명'].count()):
    str1 = train.iloc[j,2] 
    if str1[len(str1)-1:] == "우":
        if str1[:len(str1)-1] == train.iloc[j-1,2]:
            train.iloc[j,2] = str1[:len(str1)-1]
    #TIGER, KODEX,KINDEX, KBSTAR, HANARO, ARIRANG, KOSEF, 신한G, 미래G로 시작하는 종목명 탈락
    if str1[:5] == "TIGER":
        train.iloc[j,2]  ="";
    elif str1[:5] == "KODEX":
        train.iloc[j,2]  ="";
    elif str1[:6] == "KINDEX":
        train.iloc[j,2]  ="";
    elif str1[:6] == "KBSTAR":
        train.iloc[j,2]  ="";
    elif str1[:6] == "HANARO":
        train.iloc[j,2]  ="";
    elif str1[:7] == "ARIRANG":
        train.iloc[j,2]  ="";
    elif str1[:5] == "KOSEF":
        train.iloc[j,2]  ="";
    elif str1[:3] == "신한G":
        train.iloc[j,2]  ="";
    elif str1[:3] == "미래G":
        train.iloc[j,2]  ="";
    elif str1[:3] == "미래F":
        train.iloc[j,2]  ="";


In [ ]:
#중복제거
print(train['종목명'].count())
len(train['종목명'].unique())
sample = train['종목명'].unique()
print(len(sample))

# 시험용
sampleF = ['동화약품', 'KR모터스', '경방', '메리츠화재', '삼양홀딩스', '하이트진로', '유한양행', 'CJ대한통운']

In [ ]:
#train.to_csv('종목명v7.csv', index=False, encoding='euc-kr')

In [ ]:
# 크롤링 안정성을 위해 작업량 쪼개기
# 2688개를 180개씩 나누기
#1
sampleAPI1 = sample[:180]
sampleAPI2 = sample[180:360]
sampleAPI3 = sample[360:540]
#2
sampleAPI4 = sample[540:720]
sampleAPI5 = sample[720:900]
sampleAPI6 = sample[900:1080]
#3
sampleAPI7 = sample[1080:1260]
sampleAPI8 = sample[1260:1440]
sampleAPI9 = sample[1440:1620]
#4
sampleAPI10 = sample[1620:1800]
sampleAPI11 = sample[1800:1980]
sampleAPI12 = sample[1980:2160]
#5
sampleAPI13 = sample[2160:2340]
sampleAPI14 = sample[2340:2520]
sampleAPI15 = sample[2520:]


## [1]트위터 스크래핑!

### 2-1. 사전작업
 - 구글계정으로 트위터를 가입해 주세요.
 - 아래계정을 검색한뒤 점3개를 클릭해 차단(Block)해주세요
증권사 기업 레포트 봇, 바인더배송준비, 제트 09, Kmfresh19, 금융달인, KISMO, 경제/투자 유튜브 새영상 알림 봇, 증시봇, 싸주닷컴 야동, 다벗구놀기홈:kcm78.com, SKim35052643

### 2-2. 코드 작성
아래코드에서 수정할 부분들
 - 1. 크롤링을 위해 트위터(구글계정) id,pw를 적어주세요
 - 2.자신에게 맞는 크롬드라이버 위치를 적어주세요.
 - 3.자신에게 할당된 샘플넘버를 맞춰주세요.

In [ ]:
snsresult = pd.DataFrame(columns=['종목','내용'])
errresult = pd.DataFrame(columns=['에러종목','에러'])
num= 0
errnum=0
#1. 크롤링을 위해 트위터 id,pw를 적어주세요(제가 구글로그인으로 해서 구글계정으로 해주세요)
input_id = "dek930710";
input_pw = "indek132!@";

#페이지이동 

# 2.자신에게 맞는 크롬드라이버 위치를 적어주세요
driver=wb.Chrome(r'C:\Users\SM2113\크롤링\chromedriver')
driver.get("https://twitter.com/search?q=sk%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&src=typed_query&f=live")
#로그인
btn_lgnpage = driver.find_element_by_css_selector("#layers > div > div:nth-child(1) > div > div > div > div > div > div > div:nth-child(1)")
btn_lgnpage.click()
time.sleep(0.3)
id = driver.find_element_by_css_selector("#react-root > div > div > div.css-1dbjc4n.r-13qz1uu.r-417010 > main > div > div > div.css-1dbjc4n.r-13qz1uu > form > div > div:nth-child(6) > label > div > div.css-1dbjc4n.r-18u37iz.r-16y2uox.r-1wbh5a2.r-1wzrnnt.r-1udh08x.r-xd6kpl.r-1pn2ns4.r-ttdzmv > div > input")
id.send_keys(input_id)
pw = driver.find_element_by_css_selector("#react-root > div > div > div.css-1dbjc4n.r-13qz1uu.r-417010 > main > div > div > div.css-1dbjc4n.r-13qz1uu > form > div > div:nth-child(7) > label > div > div.css-1dbjc4n.r-18u37iz.r-16y2uox.r-1wbh5a2.r-1wzrnnt.r-1udh08x.r-xd6kpl.r-1pn2ns4.r-ttdzmv > div > input")
pw.send_keys(input_pw)
btn_lgn = driver.find_element_by_css_selector("#react-root > div > div > div.css-1dbjc4n.r-13qz1uu.r-417010 > main > div > div > div.css-1dbjc4n.r-13qz1uu > form > div > div:nth-child(8) > div > div")
btn_lgn.click()
time.sleep(0.5)
for k in range(1):# 3.자신에게 할당된 샘플넘버를 맞춰주세요
#     if(k == 0):
#         jongmok = sampleAPI1
#     elif(k == 1):
#         jongmok = sampleAPI2
#     elif(k == 2):
#         jongmok = sampleAPI3
    jongmok = sampleAPI6
    for jong in jongmok: 
        # 종목 검색
        try :
            driver.get("https://twitter.com/search?q="+jong+"&src=recent_search_click&f=live")
            time.sleep(1.0)
        except ValueError :
            errcode = "종목명 : ",jong, " driver.get 에러"
            errresult.loc[errnum] = [jong,errcode]
            errnum = errnum+1
        for j in range(10): # 종목당 크롤링 시도 횟수
            try :
                for d in range(30): # 스크롤 키다운
                    body=driver.find_element_by_tag_name("body")
                    body.send_keys(Keys.PAGE_DOWN)
                    time.sleep(0.2)
                time.sleep(0.2)

                #body가져오기
                body=driver.find_element_by_tag_name("body")
                time.sleep(2.5)
                result = driver.find_elements_by_css_selector('#react-root > div > div > div.css-1dbjc4n.r-18u37iz.r-13qz1uu.r-417010 > main > div > div > div > div > div > div > div > div > section > div > div > div')
                time.sleep(1)
            except ValueError :
                errcode = "종목명 : ",jong, " body not found에러"
                errresult.loc[errnum] = [jong,errcode]

            for i in range(len(result)): # 데이터 프레임에 저장
                try :
                    snsresult.loc[num]=[ jong, result[i].text.strip()]
                except ValueError :
                    errcode = "종목명 : ",jong,"  ",j,"번째 크롤링 시도중 elements not found에러"
                    errresult.loc[errnum] = [jong,errcode]
                    errnum = errnum+1
                    #selector로 가져온값 저장
                print(snsresult.loc[num])
                num = num+1





In [ ]:
#크롤링한 데이터 저장(동적저장x)
snsresult.to_csv('snsresult66.csv', index=False, encoding='utf-8-sig')
errresult.to_csv('errresult66.csv', index=False, encoding='utf-8-sig')

## [2]트위터 API 활용

In [ ]:
#트위터 api인증키 (트위터디벨롭에서 받아몽)

twitter_consumer_key = "XNcqVsLod2TYdqijZT5bzfS3x"
twitter_consumer_secret = "j6kKFtFp0fpegZ4Hzifl52E8njyCuEksGpqenbAQkOGHoj09sI"  
twitter_access_token = "1422812027811090433-vYxgHirfscvj6EiaKotxVFn7DUrsLz"
twitter_access_secret = "x8b3gd0gQDulNzzrnnk5WKNzbTyvi1i1eU2grg5dVIjLO"

# twitter 라이브러리 임폴트



twitter_api = twitter.Api(consumer_key=twitter_consumer_key,
                          consumer_secret=twitter_consumer_secret, 
                          access_token_key=twitter_access_token, 
                          access_token_secret=twitter_access_secret)

#### 특정 단어로 검색하기

In [ ]:
# 최신순으로 검색
# GetSearch(term 검색명, count 가져오는 횟수)
#               #검색명 해서 태그로 가져올수 있음

#jongmook = {} *종목은 investing이라는 주식종목 검색기에서 크롤링
#for i in range(len(jongmook))
jongmook = "SK하이닉스"
statuses = twitter_api.GetSearch(term=jongmook, count=100
                                )
for status in statuses:
    print(status.text)


#### 유저아이디로 검색

In [ ]:
account = "@pogba9999" # 주식종목 추천봇 ID

statuses = twitter_api.GetUserTimeline(screen_name=account, 
                count=200, include_rts=True, exclude_replies=False)

for status in statuses:
    print(status.text.encode('utf-8'))

### 사용된 태그가 많은 순으로 출력
- Counter()를 활용해서 가장 많이 등장한 해시태그 순으로 출력.


In [ ]:
from collections import Counter
query = "#sk하이닉스"
statuses = twitter_api.GetSearch(term=query, count=100)
result = []
for status in statuses:
    for tag in status.hashtags:
        result.append(tag.text)
        
Counter(result).most_common(20)

#### 스트리밍 GetStreamFilter()
 - 특정 키워드를 지정해놓으면, 그 키워드가 포함된 트윗을 실시간으로 수집하는 거다.

In [ ]:
import json
query = ["주식종목 추천"]
output_file_name = "stream_result.txt"
with open(output_file_name, "w", encoding="utf-8") as output_file:
    stream = twitter_api.GetStreamFilter(track=query)
    while True:
        for tweets in stream:
            tweet = json.dumps(tweets, ensure_ascii=False)
            print(tweet, file=output_file, flush=True)

# [3]페이스북 스크래핑!

In [ ]:
driver=wb.Chrome(r'C:\Users\SM2113\크롤링\chromedriver')
driver.get("https://m.facebook.com/search/posts/?q=%EC%98%81%EC%9B%85%EB%AC%B8&source=filter&isTrending=0&tsid")

html = driver.page_source
soup = bs(html,'html.parser')

## 로그인하기 
btn_lgnpage = driver.find_element_by_css_selector("#root > div:nth-child(1) > div > div > div:nth-child(1) > div > a._7nyk._7nyj > div")
btn_lgnpage.click()
id = driver.find_element_by_css_selector("#m_login_email")
id.send_keys("+821073372836")
pw = driver.find_element_by_css_selector("#m_login_password")
pw.send_keys("indek132!@")
btn_lgn = driver.find_element_by_css_selector("#login_password_step_element > button")
btn_lgn.click()
time.sleep(1.5)
driver.get("https://m.facebook.com/search/posts/?q=삼성전자&source=filter&isTrending=0&tsid")
time.sleep(1.5)
body=driver.find_element_by_tag_name("body")
for i in range(200):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
nnn = driver.find_elements_by_css_selector('div._5rgt._5nk5._5msi p')

for i in range(len(nnn)):
     nnn[i] = nnn[i].text.strip()
nnn

